In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
import time


In [3]:
url = 'http://192.168.1.5:5000'
volume = {'volume':5}

In [4]:
r = requests.post(url + '/get_status', json = {'name':'1cc_2'})

In [5]:
r.content

b'{"remaining_volume":null,"syringe_loaded":false}\n'

In [6]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [7]:
#jubilee.home_all()

In [7]:
deck = jubilee.load_deck('lab_automation_deck.json')

# load a well plate 

In [8]:
samples = jubilee.load_labware('falcon_48_wellplate_1500ul.json', 0)
samples.manual_offset([(21.4, 82.9), (134.1, 83.2), (133.9, 12.8)])

New manual offset applied to falcon_48_wellplate_1500ul


In [9]:
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
#stocks.manual_offset()

In [10]:
syringe_10 = syringe.HTTPSyringe.from_config(1, "src/science_jubilee/tools/configs/10cc_syringe.json")

In [11]:
syringe_1_1 = syringe.HTTPSyringe.from_config(2, "src/science_jubilee/tools/configs/1cc_1_syringe.json")

In [12]:
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "src/science_jubilee/tools/configs/1cc_2_syringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "src/science_jubilee/tools/configs/1cc_3_syringe.json")

In [13]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [14]:
syringe_1_1.status()

{'remaining_volume': None, 'syringe_loaded': False}

In [59]:
jubilee.pickup_tool(syringe_1_1)

In [20]:
syringe_1_1.set_pulsewidth(1850)

In [21]:
syringe_1_1.load_syringe(0, 1850)

Loaded syringe, remaining volume 0 uL


## Manual syringe load:

In [21]:
syringe_1_1.set_pulsewidth(syringe_1_1.empty_position-1)

In [22]:
aspirate = samples[0].bottom(+200)
aspirate_vol = 995 #- syringe_1_1.remaining_volume
#aspirate_vol = 9995
syringe_1_1.aspirate(aspirate_vol, aspirate)

In [32]:
syringe_1_1.set_pulsewidth(1550)

In [96]:
## Run dispense experiment 

In [97]:
# time step sequence:
"""
3 samples per each:
- 1
- 10
- 50
- 100
- 200
- 300
"""

#### Modify here
test_step = 300

######Don't touch here ######
syringe_1.set_pulsewidth(1300)
syringe_pulsewidth = 1300

jubilee.pickup_tool(syringe_1)
for i in range(1):
    dispense_location = samples[i].bottom(+25)
    new_pulsewidth = syringe_pulsewidth + test_step
    syringe_pulsewidth = new_pulsewidth
    jubilee.move_to(z = jubilee.deck.safe_z)

    jubilee.move_to(x = dispense_location.point[0], y = dispense_location.point[1], wait = True)
    jubilee.move_to(z = dispense_location.point[2], wait = True)
    syringe_1.set_pulsewidth(new_pulsewidth)
    time.sleep(10)


## Check calibration

In [30]:
#syringe_1_1.dispense(30, samples[0])

In [31]:
dispense_vol = 100

#jubilee.pickup_tool(syringe_1)
for i in range(3):
    dispense_location = samples[i].bottom(+30)
    syringe_1_1.dispense(dispense_vol, dispense_location)
    time.sleep(7)

In [19]:
dispense_location = samples[1].bottom(+25)

In [20]:
dispense_location

Location(point=(37.499996100392444, 82.85570212147425, 27.41), labware=Well A2 form Falcon 48 Well Plate 1500 µL on slot 0)

In [27]:
samples[1]

Well A2 form Falcon 48 Well Plate 1500 µL on slot 0

In [50]:
jubilee.park_tool()